In [1]:
# imports
import json
import pandas as pd
import numpy as np
import random
import datetime
import sklearn
import math
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from time import time
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
# reading data from bhopal
df = pd.read_excel('./LevelOneCleaning/Bhopal.xlsx', sheet_name='Orders')
l = df.to_dict('records')
orders = []
for dic in l:
	orders.append(dic)
ordersBhopal = []
for dic in l:
    ordersBhopal.append(dic)

In [ ]:
#reading data from Mumbai
df2 = pd.read_excel('./LevelOneCleaning/Mumbai.xlsx', sheet_name='Orders')
l2 = df2.to_dict('records')
for dic in l2:
    orders.append(dic)
    
ordersMumbai = []
for dic in l:
    ordersMumbai.append(dic)

In [ ]:
def makeFeaturesFromItems(oi):
    res = json.loads(oi)
    a = res['order_menu']
    for y in a:
        l1 = [y['menu_description'], y['quantity'], y['order_line_total'], y['menu_code']]
        if(y['menu_code'][0:3] == 'BEV' or y['menu_code'][0:3] == 'DIP'):
            continue
        if y['menu_code'][0:3] in dic:
            dic[y['menu_code'][0:3]] += int(y['quantity'])
        else:
            dic[y['menu_code'][0:3]] = int(y['quantity'])
    return dic

In [ ]:
def getOneHotEncodingRestaurant(dp):
    restaurants = ['dpi63827', 'dpi63919', 'dpi64196','dpi66154','dpi66221','dpi66313','dpi66373','dpi66449','dpi66576','dpi66653','dpi67072']
    l = []
    for i in range(0, len(restaurants)):
        if(restaurants[i] == dp):
            l.append(1)
        else:
            l.append(0)
    return l

In [ ]:
def getItemsFeatures(it):
    itd = makeFeaturesFromItems(it)
    item_types = ['BRD','CAK','DST','NVP','PIZ','SID','TAC','VGP']
    l = []
    for i in range(0, len(item_types)):
        l.append(0)
    for key in itd.keys():
        for i in range(0, len(item_types)):
            if(item_types[i] == key):
                l[i] = itd[key]
                break
    return l

In [ ]:
total_slots = 24
def getTimeIndex(dt):
	current = datetime.datetime(2011, 1, 1)
	t1 = current.time()
	t2 = dt.time()
	for i in range(0, total_slots):
		current = current + datetime.timedelta(minutes = 60)
		if(current.time() > t2):
			return i
	return total_slots-1

In [ ]:
def getOneHotEncodingTimeSlot(ts):
    l = []
    for i in range(0, 24):
        l.append(0)
    l[ts] = 1
    return l

In [ ]:
def get_preparation_time(order):
	dt1 = datetime.datetime.strptime(order['ORDER_MAKELINE_DATE_TIME'], "%Y-%m-%d %H:%M:%S")
	dt2 = datetime.datetime.strptime(order['KITCHEN_DISPLAY_TIME'], "%Y-%m-%d %H:%M:%S")
	diff = dt1 - dt2
	return diff.total_seconds()

In [ ]:
# features for Bhopal Model
# 
threshold = 30 # minutes for preparation time (threshold)
maxDifference = 1 # minute difference (abs) between predicted time and real time to assume prediction correct
splitMethod = 'random' # random, weekdays, weekends, mix
X_train, X_test, y_train, y_test = [], [], [], []
X = []
y = []
for order in ordersBhopal:
    features = []
    features = getItemsFeatures(order['ORDER_ITEMS'])
    features = features + getOneHotEncodingRestaurant(order['STORE_ID'])
    features = features + getOneHotEncodingTimeSlot(getTimeIndex(datetime.datetime.strptime(order['ORDER_RECEIVED_DATE_TIME'], "%Y-%m-%d %H:%M:%S")))
#     if(get_preparation_time(order) > (threshold-9)*60): #removing inputs 
#         continue
    X.append(features)
    y.append(540+get_preparation_time(order))
    if((order['ORDER_DATE'] >='2021-12-20' and order['ORDER_DATE'] <= '2021-12-24') or (order['ORDER_DATE'] >='2021-12-27' and order['ORDER_DATE'] <= '2021-12-29')):
        X_train.append(features)
        y_train.append(540+get_preparation_time(order))
    elif(order['ORDER_DATE'] =='2021-12-30'):
        X_test.append(features)
        y_test.append(540+get_preparation_time(order))

In [78]:
len(X)

18309

In [ ]:
if splitMethod == 'random':
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [85]:
mlp = MLPClassifier(hidden_layer_sizes=(80, 30, 10), activation='relu', max_iter=1000, verbose=True, learning_rate='constant', learning_rate_init=0.01)

In [86]:
#activation: relu, logistic, 

In [89]:
mlp.fit(X_train, y_train)
print(mlp.best_loss_)

Iteration 1, loss = 8.94234618
Iteration 2, loss = 6.23212646
Iteration 3, loss = 5.99764360
Iteration 4, loss = 5.88225634
Iteration 5, loss = 5.82635889
Iteration 6, loss = 5.79713412
Iteration 7, loss = 5.78075823
Iteration 8, loss = 5.77066400
Iteration 9, loss = 5.76381215
Iteration 10, loss = 5.75891112
Iteration 11, loss = 5.75542445
Iteration 12, loss = 5.75267255
Iteration 13, loss = 5.75062544
Iteration 14, loss = 5.74862906
Iteration 15, loss = 5.74736932
Iteration 16, loss = 5.74591542
Iteration 17, loss = 5.74519347
Iteration 18, loss = 5.74456708
Iteration 19, loss = 5.74420422
Iteration 20, loss = 5.74377744
Iteration 21, loss = 5.74344878
Iteration 22, loss = 5.74311662
Iteration 23, loss = 5.74311713
Iteration 24, loss = 5.74291607
Iteration 25, loss = 5.74304818
Iteration 26, loss = 5.74287426
Iteration 27, loss = 5.74272567
Iteration 28, loss = 5.74267729
Iteration 29, loss = 5.74267284
Iteration 30, loss = 5.74251606
Iteration 31, loss = 5.74233710
Iteration 32, los

In [90]:
mlp.out_activation_

'softmax'

In [91]:
predictions = mlp.predict(X_train)

In [92]:
len(y_test)

3662

In [98]:
mlp.predict_log_proba(X)

array([[-6.86428495, -7.47516641, -7.1210821 , ..., -9.65384623,
        -9.60329861, -9.60688109],
       [-6.86428495, -7.47516641, -7.1210821 , ..., -9.65384623,
        -9.60329861, -9.60688109],
       [-6.86428495, -7.47516641, -7.1210821 , ..., -9.65384623,
        -9.60329861, -9.60688109],
       ...,
       [-6.86428495, -7.47516641, -7.1210821 , ..., -9.65384623,
        -9.60329861, -9.60688109],
       [-6.86428495, -7.47516641, -7.1210821 , ..., -9.65384623,
        -9.60329861, -9.60688109],
       [-6.86428495, -7.47516641, -7.1210821 , ..., -9.65384623,
        -9.60329861, -9.60688109]])

In [96]:
mse = sklearn.metrics.mean_squared_error(y_train, predictions)  
rmse = math.sqrt(mse)  
rmse

372.83643981706683

In [94]:
7cnt=0
for x, y in zip(y_test, predictions):
    if(abs(x-y) >= maxDifference*60):
        cnt+=1
#         print(x, y, x-y)
accuracy = (len(y_test)-cnt)/len(y_test)

print("Accuracy: ", accuracy)
print('total : ', len(y_test))
print("correct: ", cnt)



Accuracy:  0.5608956854178044
total :  3662
correct:  1608


In [ ]:
rmse

In [ ]:
# random forest
from sklearn.ensemble import RandomForestRegressor 

scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

regressor = RandomForestRegressor()
regressor.fit(X_train, y_train)
predictions = regressor.predict(X_train)

In [ ]:
mse = sklearn.metrics.mean_squared_error(y_train, predictions)  
rmse = math.sqrt(mse)  
rmse

In [14]:
def getRMSE(actual, predictions):
    mse = sklearn.metrics.mean_squared_error(actual, predictions)
    return math.sqrt(mse)

In [15]:
def ModelResults(X_train, X_test, y_train, y_test, model, isNormalizationRequired=True):
    if(isNormalizationRequired):
        scaler = StandardScaler()
        scaler.fit(X_train)

        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)
        
    if(model=='NN'):
        hidden_layer_sizes=(80, 30, 10)
        activation='relu'
        print("Neural Network model")
        print("Hidden layers: ", hidden_layer_sizes)
        print("Activation function: ", activation)
        mlp = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, activation=activation, max_iter=1000, verbose=True, learning_rate='constant', learning_rate_init=0.01)
        mlp.fit(X_train, y_train)
        
        predictions = mlp.predict(X_train)
        print("RMSE score on validation set: ", getRMSE(y_train, predictions))
        
        predictions = mlp.predict(X_test)
        print("RMSE score on test set: ", getRMSE(y_test, predictions))
        
    elif(model=='RF'):
        print("Random Forest Model")
        regressor = RandomForestRegressor()
        regressor.fit(X_train, y_train)
        
        predictions = regressor.predict(X_train)
        print("RMSE score on validation set: ", getRMSE(y_train, predictions))
        
        predictions = regressor.predict(X_test)
        print("RMSE score on test set: ", getRMSE(y_test, predictions))
    else:
        print("Model not implemented yet")